In [9]:
import os
import re
import torch
import numpy as np
from train import NOELSTM

# Load data
theta = np.load("theta.npy")
omega = np.load("omega.npy")
u = np.load("u.npy")

# Normalize
theta_mean, theta_std = theta.mean(), theta.std()
omega_mean, omega_std = omega.mean(), omega.std()
u_mean, u_std = u.mean(), u.std()

theta_norm = (theta - theta_mean) / theta_std
omega_norm = (omega - omega_mean) / omega_std
u_norm = (u - u_mean) / u_std

def create_sequences(theta, omega, u, seq_len):
    X, Y = [], []
    for i in range(len(theta) - seq_len):
        x_seq = np.stack([theta[i:i+seq_len], omega[i:i+seq_len], u[i:i+seq_len]], axis=1)
        y_target = [theta[i+seq_len], omega[i+seq_len]]
        X.append(x_seq)
        Y.append(y_target)
    return np.array(X), np.array(Y)

def unnormalize(preds):
    preds_unnorm = np.empty_like(preds)
    preds_unnorm[:, 0] = preds[:, 0] * theta_std + theta_mean
    preds_unnorm[:, 1] = preds[:, 1] * omega_std + omega_mean
    return preds_unnorm

def print_metrics(pred, true):
    rms = np.sqrt(np.mean((pred - true)**2))
    deg = rms / (2 * np.pi) * 360
    nrms = rms / np.std(true) * 100
    return rms, deg, nrms

model_stats = []

model_dir = "new_models"
model_files = [f for f in os.listdir(model_dir) if f.startswith("model_hs") and f.endswith(".pt")]

for model_file in model_files:
    path = os.path.join(model_dir, model_file)

    # Extract hyperparameters
    filename_no_ext = os.path.splitext(model_file)[0]  # "best_model_..."
    match = re.search(
        r"hs(\d+)_lr([\d.]+)_nl(\d+)_bs(\d+)_sl(\d+)_ep(\d+)_bi(\d+)_do([\d.]+)", filename_no_ext
    )
    if not match:
        print(f"Skipping {model_file} (couldn't parse hyperparameters).")
        continue

    hs, lr, nl, bs, sl, ep, bi, do = match.groups()
    hs, nl, sl = int(hs), int(nl), int(sl)
    bi = bool(int(bi))
    do = float(do)

    # Prepare data
    X, Y = create_sequences(theta_norm, omega_norm, u_norm, sl)
    X_tensor = torch.tensor(X).double()
    Y_tensor = torch.tensor(Y).double()

    # Train/val split
    n = len(X_tensor)
    train_end = int(0.8 * n)
    val_end = int(0.9 * n)
    
    X_train, Y_train = X_tensor[:train_end], Y_tensor[:train_end]
    X_val, Y_val = X_tensor[train_end:val_end], Y_tensor[train_end:val_end]
    X_test, Y_test = X_tensor[val_end:], Y_tensor[val_end:]

    # Load model
    model = NOELSTM(input_size=3, hidden_size=hs, num_layers=nl, 
                    output_size=2, bidirectional=bi, dropout=do)
    try:
        model.load_state_dict(torch.load(path, map_location='cpu'))
    except Exception as e:
        print(f"Error loading {model_file}: {e}")
        continue

    model.eval()
    with torch.no_grad():
        train_pred = model(X_train).numpy()
        val_pred = model(X_val).numpy()

    # Unnormalize
    train_pred = unnormalize(train_pred)
    val_pred = unnormalize(val_pred)
    Y_train_true = unnormalize(Y_train.numpy())
    Y_val_true = unnormalize(Y_val.numpy())

    # Evaluate
    train_rms, train_deg, train_nrms = print_metrics(train_pred[:, 0], Y_train_true[:, 0])
    val_rms, val_deg, val_nrms = print_metrics(val_pred[:, 0], Y_val_true[:, 0])

    model_stats.append({
        "model_file": model_file,
        "train_rms": train_rms,
        "train_deg": train_deg,
        "train_nrms": train_nrms,
        "val_rms": val_rms,
        "val_deg": val_deg,
        "val_nrms": val_nrms
    })

    # Per-model print
    print(f"\n=== {model_file} ===")
    print(f"Train RMS: {train_rms:.5f} rad | {train_deg:.2f}° | NRMS: {train_nrms:.2f}%")
    print(f"Val   RMS: {val_rms:.5f} rad | {val_deg:.2f}° | NRMS: {val_nrms:.2f}%")

# Top 5 by Validation RMS (Theta)
top_5 = sorted(model_stats, key=lambda x: x['val_rms'])[:5]

print("\n=== Top 5 Models by Validation RMS (Theta) ===")
for i, stat in enumerate(top_5, 1):
    print(f"\n#{i}: {stat['model_file']}")
    print(f"  Train RMS: {stat['train_rms']:.5f} rad | {stat['train_deg']:.2f}° | NRMS: {stat['train_nrms']:.2f}%")
    print(f"  Val   RMS: {stat['val_rms']:.5f} rad | {stat['val_deg']:.2f}° | NRMS: {stat['val_nrms']:.2f}%")



=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi0_do0.2.pt ===
Train RMS: 0.00229 rad | 0.13° | NRMS: 0.49%
Val   RMS: 0.00316 rad | 0.18° | NRMS: 0.61%

=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi0_do0.5.pt ===
Train RMS: 0.00160 rad | 0.09° | NRMS: 0.34%
Val   RMS: 0.00200 rad | 0.11° | NRMS: 0.39%

=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi0_do0.pt ===
Train RMS: 0.00167 rad | 0.10° | NRMS: 0.36%
Val   RMS: 0.00226 rad | 0.13° | NRMS: 0.44%

=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi1_do0.2.pt ===
Train RMS: 0.00206 rad | 0.12° | NRMS: 0.44%
Val   RMS: 0.00291 rad | 0.17° | NRMS: 0.56%

=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi1_do0.5.pt ===
Train RMS: 0.00160 rad | 0.09° | NRMS: 0.34%
Val   RMS: 0.00241 rad | 0.14° | NRMS: 0.47%

=== model_hs16_lr0.01_nl1_bs128_sl10_ep200_bi1_do0.pt ===
Train RMS: 0.00176 rad | 0.10° | NRMS: 0.37%
Val   RMS: 0.00220 rad | 0.13° | NRMS: 0.43%

=== model_hs16_lr0.01_nl1_bs128_sl15_ep200_bi0_do0.2.pt ===
Train RMS: 0.00174 rad | 0.10° | NRMS

In [10]:
# Top 5 by Validation RMS (Theta)
top_5 = sorted(model_stats, key=lambda x: x['train_rms'])[:5]

print("\n=== Top 5 Models by Validation RMS (Theta) ===")
for i, stat in enumerate(top_5, 1):
    print(f"\n#{i}: {stat['model_file']}")
    print(f"  Train RMS: {stat['train_rms']:.5f} rad | {stat['train_deg']:.2f}° | NRMS: {stat['train_nrms']:.2f}%")
    print(f"  Val   RMS: {stat['val_rms']:.5f} rad | {stat['val_deg']:.2f}° | NRMS: {stat['val_nrms']:.2f}%")



=== Top 5 Models by Validation RMS (Theta) ===

#1: model_hs64_lr0.01_nl1_bs128_sl5_ep200_bi0_do0.2.pt
  Train RMS: 0.00113 rad | 0.06° | NRMS: 0.24%
  Val   RMS: 0.00138 rad | 0.08° | NRMS: 0.27%

#2: model_hs32_lr0.01_nl1_bs128_sl10_ep200_bi0_do0.5.pt
  Train RMS: 0.00122 rad | 0.07° | NRMS: 0.26%
  Val   RMS: 0.00160 rad | 0.09° | NRMS: 0.31%

#3: model_hs32_lr0.01_nl1_bs64_sl5_ep200_bi0_do0.5.pt
  Train RMS: 0.00122 rad | 0.07° | NRMS: 0.26%
  Val   RMS: 0.00152 rad | 0.09° | NRMS: 0.29%

#4: model_hs32_lr0.01_nl1_bs64_sl10_ep200_bi0_do0.pt
  Train RMS: 0.00123 rad | 0.07° | NRMS: 0.26%
  Val   RMS: 0.00146 rad | 0.08° | NRMS: 0.28%

#5: model_hs64_lr0.01_nl1_bs64_sl10_ep200_bi0_do0.2.pt
  Train RMS: 0.00124 rad | 0.07° | NRMS: 0.26%
  Val   RMS: 0.00145 rad | 0.08° | NRMS: 0.28%
